In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)
    invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
    if invalid_path in sys.path:
        sys.path.remove(invalid_path)

In [2]:
# Libraries
import os
import sys
IntialiseEnv()
import tensorflow as tf
import numpy as np
import math
import import_ipynb
from sklearn.model_selection import train_test_split
from Data_Preparation.Data_Preparation import LoadTrainDataSet

importing Jupyter notebook from D:\github\Traffic_Sign_Recognition_Detection\Source_Code\Data_Preparation\Data_Preparation.ipynb
importing Jupyter notebook from D:\github\Traffic_Sign_Recognition_Detection\Source_Code\Feature_Extraction\Segmentation.ipynb


In [3]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [5]:
class CNN_Model(tf.keras.Model):
    def __init__(self):
        super(CNN_Model, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv_layer_1 = tf.keras.layers.Conv2D(filter_size,kernel_1, strides = 1, padding='same',activation = tf.nn.relu)
        self.pool_layer_1 = tf.keras.layers.MaxPool2D(pooling)
        self.conv_layer_2 = tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pooling)
        self.conv_layer_3 = tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_3 = tf.keras.layers.MaxPool2D(pooling)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(hidden_layer, activation = tf.nn.relu)
        self.drop = tf.keras.layers.Dropout(dropout)
        self.final = tf.keras.layers.Dense(num_classes, activation = 'softmax')
    
    def call(self, input_tensor, training=False):
        x = self.conv_layer_1(input_tensor)
        x = self.pool_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.pool_layer_2(x)        
        x = self.conv_layer_3(x)
        x = self.pool_layer_3(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.drop(x)
        x = self.final(x)
        return x

In [6]:
def train(TrainingDataSet, ValidationDataSet, model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):  

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            # Training DataSet
            for x_np, y_np in TrainingDataSet:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in ValidationDataSet:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [17]:
# Hyper-parameters tuning
kernel_1 = (5,5)
kernel_2 = (3,3)
pooling = (2,2)
filter_size = 32
hidden_layer = 64
dropout = 0.2
num_classes = 43

# Configurations of batch printing
print_every = 1000
num_epochs = 10
learning_rate = 1.5e-3

In [18]:
model = CNN_Model()

def model_init_fn():
    return CNN_Model()

In [19]:
def optimizer_init_fn():
    return tf.keras.optimizers.Adam(learning_rate) 

In [20]:
def saveCNNModel():
    model.save('cnn_model.model')

In [21]:
DataSetImages, DataSetLabels = LoadTrainDataSet()

Directory:  00000
Directory:  00001
Directory:  00002
Directory:  00003
Directory:  00004
Directory:  00005
Directory:  00006
Directory:  00007
Directory:  00008
Directory:  00009
Directory:  00010
Directory:  00011
Directory:  00012
Directory:  00013
Directory:  00014
Directory:  00015
Directory:  00016
Directory:  00017
Directory:  00018
Directory:  00019
Directory:  00020
Directory:  00021
Directory:  00022
Directory:  00023
Directory:  00024
Directory:  00025
Directory:  00026
Directory:  00027
Directory:  00028
Directory:  00029
Directory:  00030
Directory:  00031
Directory:  00032
Directory:  00033
Directory:  00034
Directory:  00035
Directory:  00036
Directory:  00037
Directory:  00038
Directory:  00039
Directory:  00040
Directory:  00041
Directory:  00042


In [22]:
image_train, image_valid,label_train, label_valid = train_test_split(DataSetImages,DataSetLabels,stratify=DataSetLabels,test_size=0.2,random_state=42)

In [23]:
print(image_train.shape)
TrainingDataSet= Dataset(image_train,np.array(label_train), batch_size=64, shuffle=True)
ValidationDataSet= Dataset(image_valid,np.array(label_valid), batch_size=64, shuffle=True)
for t, (x, y) in enumerate(TrainingDataSet):
    print(t, x.shape, y.shape)
    if t > 5: break

(31367, 80, 80, 1)
0 (64, 80, 80, 1) (64,)
1 (64, 80, 80, 1) (64,)
2 (64, 80, 80, 1) (64,)
3 (64, 80, 80, 1) (64,)
4 (64, 80, 80, 1) (64,)
5 (64, 80, 80, 1) (64,)
6 (64, 80, 80, 1) (64,)


In [24]:
train(TrainingDataSet, ValidationDataSet, model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Iteration 0, Epoch 1, Loss: 3.7517614364624023, Accuracy: 1.5625, Val Loss: 3.7491939067840576, Val Accuracy: 5.049732208251953
Iteration 1000, Epoch 3, Loss: 0.3184906840324402, Accuracy: 89.47368621826172, Val Loss: 0.16566155850887299, Val Accuracy: 95.47309112548828
Iteration 2000, Epoch 5, Loss: 0.15517811477184296, Accuracy: 95.1435775756836, Val Loss: 0.10673973709344864, Val Accuracy: 97.19459533691406
Iteration 3000, Epoch 7, Loss: 0.12206931412220001, Accuracy: 96.0227279663086, Val Loss: 0.08

In [25]:
model._set_inputs(image_train,np.array(label_train))
saveCNNModel()

INFO:tensorflow:Assets written to: cnn_model.model\assets
